# G2Engine Reference

## Prepare Environment

In [ ]:
import com.senzing.g2.engine.G2Engine;
import com.senzing.g2.engine.G2JNI;
import com.senzing.g2.engine.G2ConfigMgr;
import com.senzing.g2.engine.G2ConfigMgrJNI;
import com.senzing.g2.engine.Result;

### Helper class for Json Rendering

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>org.glassfish</groupId>
    <artifactId>javax.json</artifactId>
    <version>1.1.4</version>
</dependency>

In [ ]:
import javax.json.*;
import static java.util.Collections.*;
import static javax.json.stream.JsonGenerator.PRETTY_PRINTING;

In [ ]:
public class JsonUtil {
    private static final JsonWriterFactory PRETTY_FACTORY
        = Json.createWriterFactory(singletonMap(PRETTY_PRINTING, true));
        
    private static final JsonWriterFactory UGLY_FACTORY
        = Json.createWriterFactory(emptyMap());
    
    public static String toJsonText(JsonValue val) {
        return toJsonText(val, true);
    }

    public static String toJsonText(JsonValue val, boolean prettyPrint) {
        JsonWriterFactory factory = (prettyPrint) ? PRETTY_FACTORY : UGLY_FACTORY;
        StringWriter sw = new StringWriter();
        JsonWriter writer = factory.createWriter(sw);
        writer.write(val);
        sw.flush();
        return sw.toString();
    }
    
    public static JsonObject parseJsonObject(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readObject();
  }

    public static JsonArray parseJsonArray(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readArray();
  }
    
    
}

In [ ]:
import java.util.UUID;
public static void RenderJSON(Object obj){
    String str = obj.toString();
    JsonObject json = JsonUtil.parseJsonObject(str);
    String Config = JsonUtil.toJsonText(json, false);
    UUID id = UUID.randomUUID();
    String uuid = id.toString();
    String div = "<div id=\""+ uuid +"\" style=\"height:100%; width:100%; background-color: LightCyan\"></div>";
    display(div, "text/html");
    String jav = "require([\"https://rawgit.com/caldwell/renderjson/master/renderjson.js\"], function() {document.getElementById(\'"+ uuid +"\').appendChild(renderjson("+json+"))});";
    display(jav, "application/javascript");
}

### Initialize Senzing configuration

Using environment variables and default values, create `senzingConfigJson`.
This value is used when instantiating Senzing objects.

In [ ]:
// Get variables used in constructing Senzing Engine configuration.

String configPath = System.getenv("SENZING_ETC_DIR");
if (configPath == null) {
    configPath = "/etc/opt/senzing";
}

String supportPath = System.getenv("SENZING_DATA_VERSION_DIR");
if (supportPath == null) {
    supportPath = "/opt/senzing/data";
}

String g2Path = System.getenv("SENZING_G2_DIR");
if (g2Path == null) {
    g2Path = "/opt/senzing/g2";
}

String resourcePath = g2Path + "/resources";

String sqlConnection = System.getenv("SENZING_SQL_CONNECTION");
if (sqlConnection == null) {
    sqlConnection = "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db";
}

// Construct the JSON string used for Senzing Engine configuration.

String senzingConfigJson = "{"
   + "\"PIPELINE\": {"
   +     "\"CONFIGPATH\": \"" + configPath + "\","
   +     "\"SUPPORTPATH\": \"" + supportPath + "\","
   +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
   + "},"
   + "\"SQL\": {"
   +     "\"CONNECTION\": \"" + sqlConnection + "\""
   + "}}";

RenderJSON(senzingConfigJson);

## G2Engine

### G2Engine initialization

To start using Senzing G2Engine, create and initialize an instance.
This should be done once per process.
###### Parameters

- **moduleName:** (str) A short name given to this instance of the G2Engine
  object.
- **senzingConfigJson:** (str) A JSON string containing configuration parameters.
- **verboseLogging:** (bool) A boolean which enables diagnostic logging.

Calling this function will return "0" upon success.

In [ ]:
G2Engine g2engine = new G2JNI();

String moduleName = "ExampleG2Engine";
boolean verboseLogging = true;
int return_code = g2engine.init(moduleName, senzingConfigJson, verboseLogging);
System.out.print(return_code);

#### destroy()
destroy the engine so that we can initalize differently

In [ ]:
g2engine.destroy();

### initWithConfigID

Alternatively `initWithConfigID()` can be used to specify a configuration.

###### Parameters

- **moduleName:** (str) A short name given to this instance of the G2Engine
  object.
- **senzingConfigJson:** (str) A JSON string containing configuration parameters.
- **verboseLogging:** (bool) A boolean which enables diagnostic logging.

Create G2Config so that we can use a configID

In [ ]:
String moduleName = "ExampleG2Config";
boolean verboseLogging = true;

G2ConfigMgr g2configMgr = new G2ConfigMgrJNI();
int return_code = g2configMgr.init(moduleName, senzingConfigJson, verboseLogging);
Result<Long> config = new Result<Long>();
g2configMgr.getDefaultConfigID(config);
if(return_code!=0)
    System.out.print(g2configMgr.getLastException());
else
    System.out.print(return_code);

In [ ]:
String moduleName = "ExampleG2Engine";
boolean verboseLogging = true;
int return_code = g2engine.initWithConfigID(moduleName, senzingConfigJson, config.getValue(), verboseLogging);

System.out.print(return_code);

### reinit

The `reinit()` function may be used to reinitialize the engine with a configuration

In [ ]:
String moduleName = "ExampleG2Engine";
boolean verboseLogging = true;
int return_code = g2engine.reinit(config.getValue());

System.out.print(return_code);

### primeEngine

The `primeEngine()` method may optionally be called to pre-initialize
some of the heavier weight internal resources of the G2 engine.

In [ ]:
int return_code= g2engine.primeEngine();
if(return_code!=0)
    System.out.print(g2engine.getLastException());
System.out.print(return_code)

### getActiveConfigID

Call `getActiveConfigID()` to return an identifier for the loaded
Senzing engine configuration.
The call will assign a long integer to a user-designated variable

###### Parameters

- **configID:** (long) The identifier value for the engine configuration. The result of function call is returned here

In [ ]:
Result<Long> configID = new Result<Long>();
g2engine.getActiveConfigID(configID);

System.out.print(configID.getValue());

### exportConfig

Call `exportConfig()` to retrieve your Senzing engine's configuration.
The call will assign a JSON document to a user-designated buffer,
containing all relevant configuration information
-- the function itself will return "0" upon success.
The exportConfig function accepts the following parameters as input:

###### Parameters

- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.
- **configID:** (long) The identifier value for the engine configuration. The result of function call is returned here

In [ ]:
StringBuffer response = new StringBuffer();

g2engine.exportConfig(response, configID);
RenderJSON(response)

### stats

Call `stats()` to retrieve workload statistics for the current process.
These statistics will automatically reset after retrieval.

- **respone:** (String) Object to store the output of the method. can be used to print or store for later

In [ ]:
String response = g2engine.stats();
RenderJSON(response);

### getRepositoryLastModifiedTime

Call `getRepositoryLastModifiedTime()` to obtain the last modified time of
the Senzing repository,measured in the number of seconds between the last
modified time and January 1, 1970 12:00am GMT (epoch time).
The call will assign a long integer to a user-designated buffer

###### Parameters

- **lastModifiedTime:** (long) The last modified time. The result of function
  call is returned here


In [ ]:
Result<Long> lastModifiedTime = new Result<Long>();

g2engine.getRepositoryLastModifiedTime(lastModifiedTime);

System.out.print(lastModifiedTime.getValue());

## Insert

### Insert parameters

The following variables are used as parameters to the Senzing API.

In [ ]:
String dataSourceCode1 = "TEST";
String recordID1 = "1";
String dataSourceCode2 = "TEST";
String recordID2 = "2";
String dataSourceCode3 = "TEST";
String recordID3 = "3";
String dataSourceCode4 = "TEST";
String recordID4 = "4";
String dataSourceCode5 = "TEST";
String recordID5 = "5";
String dataSourceCode6 = "TEST";
String recordID6 = "6";
String dataSourceCode7 = "TEST";
String recordID7 = "7";
String dataSourceCode8 = "TEST";
String recordID8 = "8";

String loadID = null;

### addRecord

Once the Senzing engine is initialized, use `addRecord()` to load a record
into the Senzing repository
-- `addRecord()` can be called as many times as desired and from multiple
threads at the same time.
The `addRecord()` function returns "0" upon success, and accepts four
parameters as input:

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record
  is associated with.
  This value is configurable to the system
- **recordID:** (str) The record ID, used to identify distinct records
- **jsonData:** (str) A JSON document with the attribute data for the record
- **loadID:** (str) The observation load ID for the record;
  value can be null and will default to data_source

In [ ]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"JANE\", \"NAME_LAST\": \"SMITH\", \"ADDR_TYPE\": \"HOME\", \"ADDR_LINE1\": \"653 STATE ROUTE 7\", \"ADDR_CITY\": \"FRESNO\", \"ADDR_STATE\": \"CA\", \"ADDR_POSTAL_CODE\": \"55073-1234\", \"SSN_NUMBER\": \"111-11-1111\"}";
StringBuffer response = new StringBuffer();
String dataSourceCode = dataSourceCode1;
String recordID = recordID1;

int return_code= g2engine.addRecord(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

### addRecordWithReturnedRecordID

Alternatively `addRecordWithReturnedRecordID()` can be used to return the record ID

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record
  is associated with.
  This value is configurable to the system
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.- **record_id:** (str) The record ID, used to identify distinct records
- **jsonData:** (str) A JSON document with the attribute data for the record
- **loadID:** (str) the observation load ID for the record;
  value can be null.

In [ ]:
StringBuffer responseBuffer = new StringBuffer();

int return_code = g2engine.addRecordWithReturnedRecordID(
                            dataSourceCode, 
                            responseBuffer, 
                            jsonData, 
                            loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print("New Record ID: " + responseBuffer.toString());

### addRecordWithInfo

Use if you would like to know what resolved entities were modified when
adding the new record.
It behaves identically to addRecord(),
but returns a json document containing the IDs of the affected entities.

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify distinct records
- **jsonData:** (str) A JSON document with the attribute data for the record
- **loadID:** (str) the observation load ID for the record;
  value can be null.
- **g2EngineFlags:** (long) Control flags for specifying what data about the
  entity to retrieve
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();
long g2EngineFlags = 0;
int return_code = g2engine.addRecordWithInfo(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID, 
                    g2EngineFlags,
                    response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

### addRecordWithInfoWithReturnedRecordID


###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **jsonData:** (str) A JSON document with the attribute data for the record
- **loadID:** (str) the observation load ID for the record;
  value can be null.
- **g2EngineFlags:** (long) Control flags for specifying what data about the
  entity to retrieve
- **recordResponse:** (StringBuffer) The record ID of the added record.
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();
StringBuffer recordResponse = new StringBuffer();

int return_code = g2engine.addRecordWithInfoWithReturnedRecordID(
                    dataSourceCode, 
                    jsonData,
                    loadID,
                    g2EngineFlags,
                    recordResponse,   
                    response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(record_response);
    RenderJSON(response)

## Replace

### replaceRecord

Use the `replaceRecord()` function to update or replace a record in the data
repository.
If record doesn't exist, a new record is added to the data repository.
Like the above functions, `replaceRecord()` returns "0" upon success,
and it can be called as many times as desired and from multiple threads at
the same time.

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system
- **recordID:** (str) The record ID, used to identify distinct records
- **jsonData:** (str) A JSON document with the attribute data for the record
- **loadID:** (str) the observation load ID for the record;
  value can be null.

In [ ]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"JANE\", \"NAME_LAST\": \"ADAMS\", \"ADDR_TYPE\": \"HOME\", \"ADDR_LINE1\": \"653 STATE ROUTE 7\", \"ADDR_CITY\": \"FRESNO\", \"ADDR_STATE\": \"CA\", \"ADDR_POSTAL_CODE\": \"55073-1234\"}";

int return_code= g2engine.replaceRecord(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code)

### replaceRecordWithInfo

`replaceRecordWithInfo()` is available if you would like to know what
resolved entities were modified when replacing a record.
It behaves identically to `replaceRecord()`,
but also returns a json document containing the IDs of the affected entities.

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify distinct records
- **jsonData:** (str) A JSON document with the attribute data for the record
- **loadID:** (str) the observation load ID for the record;
  value can be null.
- **g2EngineFlags:** (long) Control flags for specifying what data about the
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"JANE\", \"NAME_LAST\": \"ADAMS\", \"ADDR_TYPE\": \"HOME\", \"ADDR_LINE1\": \"653 STATE ROUTE 7\", \"ADDR_CITY\": \"FRESNO\", \"ADDR_STATE\": \"CA\", \"ADDR_POSTAL_CODE\": \"55073-1234\"}";

StringBuffer response = new StringBuffer();

int return_code= g2engine.replaceRecordWithInfo(
                            dataSourceCode, 
                            recordID, 
                            jsonData, 
                            loadID, 
                            g2EngineFlags, 
                            response);
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response)

## Re-evaluate

### reevaluateRecord

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify distinct records
- **g2EngineFlags:** (long) Control flags for outputting entities

In [ ]:
String dataSourceCode = dataSourceCode1;
String recordID = recordID1;

int return_code = g2engine.reevaluateRecord(
                            dataSourceCode, 
                            recordID, 
                            g2EngineFlags);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code);

### reevaluateRecordWithInfo

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify distinct records
- **g2EngineFlags:** (long) Control flags for outputting entities
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.reevaluateRecordWithInfo(
                            dataSourceCode, 
                            recordID, 
                            g2EngineFlags, 
                            response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

### reevaluateEntity

- **entityID:** (int) The entity ID for the entity to be analyzed
- **g2EngineFlags:** (int) Control flags for specifying what data about the entity to retrieve

Find an entity

In [ ]:
String dataSourceCode = dataSourceCode1;
String recordID = recordID1;
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(dataSourceCode, recordID, g2EngineFlags, response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();

Re-evaluate the entity.

###### Parameters

- **entityID:** (int) The entity ID for the entity to be analyzed
- **g2EngineFlags:** (long) Control flags for specifying what data about the

In [ ]:
int return_code = g2engine.reevaluateEntity(entityID, g2EngineFlags);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code);

### reevaluateEntityWithInfo

###### Parameters

- **entityID:** (int) The entity ID for the entity to be analyzed
- **g2EngineFlags:** (long) Control flags for specifying what data about the entity to retrieve
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();

return_code = g2engine.reevaluateEntityWithInfo(entityID, g2EngineFlags, response);

if(return_code!=0)
    System.out.print(return_code);
else
    RenderJSON(response);

## Redo Processing

Redo records are automatically created by Senzing when certain conditions
occur where it believes more processing may be needed.
Some examples:

- A value becomes generic and previous decisions may need to be revisited
- Clean up after some record deletes
- Detected related entities were being changed at the same time
- A table inconsistency exists, potentially after a non-graceful shutdown

First we will need to have a total of 6 data sources so let's add 5 more.

Create Record and Entity for redo

In [ ]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Lily\", \"NAME_LAST\": \"Owens\", \"SSN_NUMBER\": \"111-11-1111\"}";

String dataSourceCode = dataSourceCode4;
String recordID = recordID4;
int return_code= g2engine.addRecord(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(dataSourceCode, recordID, g2EngineFlags, response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID6 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();
System.out.println("Entity ID: " + entityID6);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

Create another Record and Entity

In [ ]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"August\", \"NAME_LAST\": \"Bauler\", \"NAME_MIDDLE\": \"E\", \"SSN_NUMBER\": \"111-11-1111\"}";

String dataSourceCode = dataSourceCode5;
String recordID = recordID5;
int return_code= g2engine.addRecord(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(dataSourceCode, recordID, g2EngineFlags, response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID7 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();
System.out.println(entityID7);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

Create another Record and Entity

In [ ]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Brian\", \"NAME_LAST\": \"Barcy\", \"NAME_MIDDLE\": \"H\", \"SSN_NUMBER\": \"111-11-1111\"}";

String dataSourceCode = dataSourceCode6;
String recordID = recordID6;
int return_code= g2engine.addRecord(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(dataSourceCode, recordID, g2EngineFlags, response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID8 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();
System.out.println(entityID8);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

Create another Record and Entity

In [ ]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Jack\", \"NAME_LAST\": \"Miller\", \"NAME_MIDDLE\": \"H\", \"SSN_NUMBER\": \"111-11-1111\"}";

String dataSourceCode = dataSourceCode7;
String recordID = recordID7;
int return_code= g2engine.addRecord(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(dataSourceCode, recordID, g2EngineFlags, response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID9 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();
System.out.println(entityID9);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

Create another Record and Entity

In [ ]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Madelyn\", \"NAME_LAST\": \"Caryn\", \"SSN_NUMBER\": \"111-11-1111\"}";

String dataSourceCode = dataSourceCode8;
String recordID = recordID8;
int return_code= g2engine.addRecord(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(dataSourceCode, recordID, g2EngineFlags, response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID10 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();
System.out.println(entityID10);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

### countRedoRecords

Once the Senzing engine is initialized, use `countRedoRecords()`
to return the remaining internally queued maintenance records in the
Senzing repository.
`countRedoRecords()` takes no arguments and returns <0 for errors.

In [ ]:
long return_code = g2engine.countRedoRecords();
System.out.print(return_code);

### getRedoRecord

Once the Senzing engine is initialized, 
use `getRedoRecord()` to retrieve the next internally queued maintenance
record into the Senzing repository
-- `getRedoRecord()` can be called as many times as desired and from multiple
threads at the same time but all threads are required to be in the same
process.
`getRedoRecord()` should not be called from multiple processes.
Unlike `processRedoRecord()`, `getRedoRecord()` does not actually process the
record.
To process the record, you would use the G2Engine `process()` function.
The `getRedoRecord()` function returns "0" upon success and an empty response
if there is nothing to do.

###### Parameters

- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.getRedoRecord(response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response)

### processWithInfo

###### Parameters

- **response:** (StringBuffer) the input record to add
- **processResponse:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.
- **g2EngineFlags:** (long) Control flags for specifying what data about the


In [ ]:
StringBuffer processResponse = new StringBuffer();

int return_code = g2engine.processWithInfo(
                            response.toString(),
                            g2EngineFlags,
                            processResponse);
                            
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response)

### process

###### Parameters

- **response:** (bytearray) the input record to process

In [ ]:
int return_code = g2engine.process(response.toString());
                            
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response)

### processRedoRecord

This processes the next redo record and returns it
(If `processRedoRecord()` "response" returns 0
and "response_bytearray" is blank then there are no more redo records to
process and if you do `countRedoRecords()` again it will return 0)
Has potential to create more redo records in certian situations.

###### Parameters

- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.processRedoRecord(response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response)

### processRedoRecordWithInfo

`processRedoRecordWithInfo()` is available if you would like to know what
resolved entities were modified when processing a redo record.
It behaves identically to `processRedoRecord()`,
but also returns a json document containing the IDs of the affected entities.
It accepts the following parameters:

###### Parameters

- **g2EngineFlags:** (long) Control flags for specifying what data about the
- **info:** (StringBuffer) Returns the "withInfo" section 
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();
StringBuffer info = new StringBuffer();

int return_code = g2engine.processRedoRecordWithInfo(
    g2EngineFlags,
    info,
    response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);
    RenderJSON(info)

## Delete

### deleteRecord

Use `deleteRecord()` to remove a record from the data repository
(returns "0" upon success);
`deleteRecord()` can be called as many times as desired and from multiple
threads at the same time.

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify distinct records
- **loadID:** (str) the observation load ID for the record;
  value can be null.

In [ ]:
String dataSourceCode = dataSourceCode1;
String recordID = recordID1;

int return_code= g2engine.deleteRecord(dataSourceCode, recordID, loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code);

### deleteRecordWithInfo

`deleteRecordWithInfo()` behaves the same as `deleteRecord()`
but also returns a json document containing the IDs of the affected entities.

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with. This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify distinct records.
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.
- **loadID:** (str) the observation load ID for the record;
  value can be null.
- **g2EngineFlags:** (long) Control flags for specifying what data about the  included in the export.

In [ ]:
String dataSourceCode = dataSourceCode2;
String recordID = recordID2;
StringBuffer response = new StringBuffer();

int return_code = g2engine.deleteRecordWithInfo(
                            dataSourceCode, 
                            recordID, 
                            loadID, 
                            g2EngineFlags, 
                            response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

Attempt to get the record again. It should error and give an output similar to "Unknown record"

In [ ]:
StringBuffer response = new StringBuffer();
int return_code = g2engine.getRecord(dataSourceCode, recordID, response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

Add record back for later API calls

In [ ]:
String dataSourceCode = dataSourceCode1;
String recordID = recordID1;

int return_code = g2engine.addRecord(dataSourceCode, recordID, jsonData, loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

String dataSourceCode = dataSourceCode2;
String recordID = recordID2;

int return_code = g2engine.addRecord(dataSourceCode, recordID, jsonData, loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

## Search

### Record search

#### getRecord

Use `getRecord()` to retrieve a single record from the data repository;
the record is assigned in JSON form to a user-designated buffer.
Once the Senzing engine is initialized,
`getRecord()` can be called as many times as desired and from multiple
threads at the same time.

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify the record for retrieval
- **g2EngineFlags:** (long) Control flags for specifying what data about the
  record to retrieve.
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
int flags = 0;
String dataSourceCode = "TEST";
String recordID = "1";
StringBuffer response = new StringBuffer();
int return_code= g2engine.getRecord(
                    dataSourceCode, 
                    recordID, 
                    flags, 
                    response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response)

### Entity search

#### getEntityByRecordID

Entity searching is a key component for interactive use of Entity Resolution
intelligence.
The core Senzing engine provides real-time search capabilities that are
easily accessed via the Senzing API.
Senzing offers methods for entity searching, all of which can be called
as many times as desired and from multiple threads at the same time
(and all of which return "0" upon success).

Use `getEntityByRecordID()` to retrieve entity data based on the ID of a
resolved identity.

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify the record for retrieval
- **g2EngineFlags:** (long) Control flags for specifying what data about the
  record to retrieve.
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
String dataSourceCode = "TEST";
String recordID = "1";

StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(
                            dataSourceCode, 
                            recordID, 
                            flags, 
                            response);

JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID1 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();

if(return_code!=0)
    RenderJSON(g2engine.getLastException());
else
    RenderJSON(response);

#### getEntityByEntityID

Use `getEntityByEntityID()` to retrieve entity data based on the ID of a
resolved identity.

###### Parameters

- **entityID:** (long) The numeric ID of a resolved entity
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later

In [ ]:
long entityID = entityID1;
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByEntityID(entityID, response);

if(return_code!=0)
    RenderJSON(g2engine.getLastException());
else
    RenderJSON(response);

#### searchByAttributes

This function is similar but preferable to the searchByAttributes() function.

Use `searchByAttributes()` to retrieve entity data based on
a user-specified set of entity attributes.

###### Parameters

- **jsonData:** (str) A JSON document with the attribute data to search for.
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later
- **g2EngineFlags:** (long) Operational flags

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.searchByAttributes(jsonData, g2EngineFlags, response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

#### getVirtualEntityByRecordID

###### Parameters

- **records:** (str) A JSON document with the datasource codes and recordID's of the records to check against
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later

In [ ]:
StringBuffer response = new StringBuffer();
String records =  "{\"RECORDS\": [{\"DATA_SOURCE\": \""+dataSourceCode1+"\", \"RECORD_ID\": \""+recordID1+"\"}, {\"DATA_SOURCE\": \""+dataSourceCode2+"\", \"RECORD_ID\": \""+recordID2+"\"}]}";

int return_code = g2engine.getVirtualEntityByRecordID(
                            records,
                            response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

### Finding Paths

First you will need to create some records so that you have some that you can
compare.
Can you see what is the same between this record and the previous one?

#### Create records for paths

In [ ]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Max\", \"NAME_LAST\": \"Miller\", \"NAME_MIDDLE\": \"W\",\"SSN_NUMBER\": \"111-11-1111\"}";

String dataSourceCode = dataSourceCode2;
String recordID = recordID2;
int return_code = g2engine.replaceRecord(dataSourceCode, recordID, jsonData, loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

Replace values for Record #3

In [ ]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Mildred\", \"NAME_LAST\": \"Miller\", \"SSN_NUMBER\": \"111-11-1111\"}";

String dataSourceCode = dataSourceCode3;
String recordID = recordID3;
int return_code = g2engine.replaceRecord(dataSourceCode, recordID, jsonData, loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

Locate "entity identifier" for Record #1

In [ ]:
String dataSourceCode = dataSourceCode1;
String recordID = recordID1;
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(
                            dataSourceCode,
                            recordID,
                            response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID2 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

Locate "entity identifier" for Record #3

In [ ]:
String dataSourceCode = dataSourceCode3;
String recordID = recordID3;
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(
                            dataSourceCode,
                            recordID,
                            response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID3 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

#### findPathByEntityID

The `findPathByEntityID()` function can be used to find single relationship paths between two entities. Paths are found using known relationships with other entities.

###### Parameters

- **entityIDStart:** (int) The entity ID for the starting entity of the search path
- **entityIDEnd:** (int) The entity ID for the ending entity of the search path
- **maxDegree:** (int) The number of relationship degrees to search
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later
- **g2EngineFlags:** (long) Control flags for specifying what data about the entity to retrieve

The functions return a JSON document that identifies the path between the
entities,
and the information on the entities in question.
The document contains a section called "ENTITY_PATHS" which gives
the path from one entity to the other.

In [ ]:
long entityIDStart = entityID2;
long entityIDEnd = entityID3;

int maxDegree = 3;

StringBuffer response = new StringBuffer();

int ret_code = g2engine.findPathByEntityID(
                            entityIDStart,
                            entityIDEnd,
                            maxDegree, 
                            g2EngineFlags, 
                            response);

if(ret_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

#### findPathByRecordID

The `findPathByRecordID()` function
can be used to find single relationship paths between two entities.
Paths are found using known relationships with other entities.

###### Parameters

- **dataSourceCodeStart:** (str) The data source for the starting entity of the
  search path
- **dataSourceCodeEnd:** (str) The data source for the ending entity of the search
  path
- **recordIDStart:** (int) The record ID for the starting entity of the search path
- **recordIDEnd:** (int) The record ID for the ending entity of the search path
- **maxDegree:** (int) The number of relationship degrees to search
- **g2EngineFlags:** (long) Control flags for specifying what data about the entity to retrieve

- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

The functions return a JSON document that identifies the path between the
entities,
and the information on the entities in question.
The document contains a section called "ENTITY_PATHS" which gives
the path from one entity to the other.

In [ ]:
String dataSourceCodeStart = dataSourceCode2;
String dataSourceCodeEnd = dataSourceCode3;
String recordIDStart = recordID2;
String recordIDEnd = recordID3;
int maxDegree = 3;

ret_code = g2engine.findPathByRecordID(
                        dataSourceCodeStart,
                        recordIDStart,
                        dataSourceCodeEnd,
                        recordIDEnd,
                        maxDegree,
                        g2EngineFlags,
                        response);

if(ret_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

#### findPathExcludingByEntityID

The `findPathExcludingByEntityID()` function can be used to find single relationship paths between two
entities.
Paths are found using known relationships with other entities.
In addition, it will find paths that exclude certain entities from being on
the path.


When excluding entities, the user may choose to either (a) strictly exclude
the entities,
or (b) prefer to exclude the entities, but still include them if no other
path is found.
By default, entities will be strictly excluded.
A "preferred exclude" may be done by specifying the
`G2_FIND_PATH_PREFER_EXCLUDE` control flag.

###### Parameters

- **entityIDStart:** (int) The entity ID for the starting entity of the search path
- **entityIDEnd:** (int) The entity ID for the ending entity of the search path
- **maxDegree:** (int) The number of relationship degrees to search
- **excludedEntities:** (str) Entities that should be avoided on the path
  (JSON document)
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later
- **g2EngineFlags:** (long) Operational flags

In [ ]:
int maxDegree = 4;
String excludedEntities = new String("{\"ENTITIES\":[{\"ENTITY_ID\":\"1\"}]}");

StringBuffer response = new StringBuffer();

long entityIDStart = entityID2;
long entityIDEnd = entityID3;

int return_code = g2engine.findPathExcludingByEntityID(
                            entityIDStart,
                            entityIDEnd,
                            maxDegree,
                            excludedEntities,
                            response);

if(ret_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

#### findPathExcludingByRecordID

The `findPathExcludingByRecordID()`
function can be used to find single relationship paths between two
entities.
Paths are found using known relationships with other entities.
In addition, it will find paths that exclude certain entities from being on
the path.

When excluding entities, the user may choose to either (a) strictly exclude
the entities,
or (b) prefer to exclude the entities, but still include them if no other
path is found.
By default, entities will be strictly excluded.
A "preferred exclude" may be done by specifying the
`G2_FIND_PATH_PREFER_EXCLUDE` control flag.

###### Parameters

- **dataSourceCodeStart:** (str) The data source for the starting entity of the
  search path
- **dataSourceCodeEnd:** (str) The data source for the ending entity of the search
  path
- **recordIDStart:** (str) The record ID for the starting entity of the search path
- **recordIDEnd:** (str) The record ID for the ending entity of the search path
- **maxDegree:** (int) The number of relationship degrees to search
- **excludedEntities:** (str) Entities that should be avoided on the path
  (JSON document)
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.
- **g2EngineFlags:** (long) Operational flags


In [ ]:
String dataSourceCodeStart = dataSourceCode2;
String dataSourceCodeEnd = dataSourceCode3;
String recordIDStart = recordID2;
String recordIDEnd = recordID3;
StringBuffer response = new StringBuffer();

String excludedRecords = new String("{\"RECORDS\":[{\"RECORD_ID\":\"1\",\"DATA_SOURCE\":\"TEST\"}]}");

return_code = g2engine.findPathExcludingByRecordID(
                        dataSourceCodeStart,
                        recordIDStart,
                        dataSourceCodeEnd,
                        recordIDEnd,
                        maxDegree,
                        excludedRecords,
                        response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

### Finding Paths with Required Sources

#### findPathIncludingSourceByEntityID

The `findPathIncludingSourceByEntityID()` function
can be used to find single relationship paths between two entities.
In addition, one of the enties along the path must include a specified data
source.

Specific entities may also be excluded, using the same methodology as the
`findPathExcludingByEntityID()` and `findPathExcludingByRecordID()`
functions use.

###### Parameters

- **entityIDStart:** (int) The entity ID for the starting entity of the search path
- **entityIDEnd:** (int) The entity ID for the ending entity of the search path
- **maxDegree:** (int) The number of relationship degrees to search
- **excludedEntities:** (str) Entities that should be avoided on the path
  (JSON document)
- **requiredDsrcs:** (str) Entities that should be avoided on the path
  (JSON document)
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later
- **g2EngineFlags:** (long) Operational flags


In [ ]:
int maxDegree = 4;
String excludedEntities = new String("{\"ENTITIES\":[{\"ENTITY_ID\":\"1\"}]}");
String requiredDsrcs = new String("{\"DATA_SOURCES\":[\"TEST\"]}");

StringBuffer response = new StringBuffer();

int ret_code = g2engine.findPathIncludingSourceByEntityID(
                            entityIDStart,
                            entityIDEnd,
                            maxDegree,
                            excludedEntities,
                            requiredDsrcs,
                            response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

#### findPathIncludingSourceByRecordID


The 
`findPathIncludingSourceByRecordID()` function
can be used to find single relationship paths between two entities.
In addition, one of the enties along the path must include a specified data
source.

Specific entities may also be excluded, using the same methodology as the
`findPathExcludingByEntityID()` and `findPathExcludingByRecordID()`
functions use.

###### Parameters

- **dataSourceCodeStart:** (str) The data source for the starting entity of the
  search path
- **dataSourceCodeEnd:** (str) The data source for the ending entity of the search
  path
- **recordIDStart:** (str) The record ID for the starting entity of the search
  path
- **recordIDEnd:** (str) The record ID for the ending entity of the search path
- **maxDegree:** (int) The number of relationship degrees to search
- **excludedEntities:** (str) Entities that should be avoided on the path
  (JSON document)
- **requiredDsrcs:** (str) Entities that should be avoided on the path
  (JSON document)
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later
- **g2EngineFlags:** (long) Operational flags


In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.findPathIncludingSourceByRecordID(
                            dataSourceCodeStart,
                            recordIDStart,
                            dataSourceCodeEnd,
                            recordIDEnd,
                            maxDegree,
                            excludedEntities,
                            requiredDsrcs,
                            response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

### Finding Networks

The `findNetworkByEntityID()` and `findNetworkByRecordID()` functions
can be used to find all entities surrounding a requested set of entities.
This includes the requested entities, paths between them, and relations to
other nearby entities.

Entities can be searched for by either Entity ID or by Record ID,
depending on which function is chosen.

These functions have the following parameters:

- **entityList:** (str) A list of entities, specified by Entity ID
  (JSON document)
- **recordList:** (str) A list of entities, specified by Record ID
  (JSON document)
- **maxDegree:** (int) The maximum number of degrees in paths between search
  entities
- **buildoutDegree:** (int) The number of degrees of relationships to show around
  each search entity
- **maxEntities:** (int) The maximum number of entities to return in the
  discovered network
- **flags:** (int) Control flags for specifying what data about the
  entity to retrieve.
- **response:** (StringBuffer) A memory buffer for returning the response
  document; if an error occurred, an error response is stored here.
  
They also have various arguments used to return response documents

The functions return a JSON document that identifies the path between the
each set of search entities (if the path exists), and the information on the
entities in question (search entities, path entities, and build-out entities.

#### findNetworkByEntityID

The `findNetworkByEntityID()` function
can be used to find all entities surrounding a requested set of entities.
This includes the requested entities, paths between them, and relations to
other nearby entities.

###### Parameters

- **maxDegree:** (int) The maximum number of degrees in paths between search
  entities
- **buildoutDegree:** (int) The number of degrees of relationships to show around
  each search entity
- **maxEntities:** (int) The maximum number of entities to return in the
  discovered network
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later

The functions return a JSON document that identifies the path between the
each set of search entities (if the path exists), and the information on the
entities in question (search entities, path entities, and build-out entities.

In [ ]:
StringBuffer response = new StringBuffer();
int maxDegree = 2;
int buildoutDegree = 1;
int maxEntities = 12;
String entityList = "{\"ENTITIES\": [{\"ENTITY_ID\":"+entityID+"}, {\"ENTITY_ID\":"+entityID2+"}]}";

int return_code = g2engine.findNetworkByEntityID(
                            entityList, 
                            maxDegree, 
                            buildoutDegree, 
                            maxEntities,
                            response);
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

#### findNetworkByRecordID

The `findNetworkByRecordID()` function
can be used to find all entities surrounding a requested set of entities.
This includes the requested entities, paths between them, and relations to
other nearby entities.

###### Parameters

- **recordList:** (str) A list of entities, specified by Record ID
  (JSON document)
- **maxDegree:** (int) The maximum number of degrees in paths between search
  entities
- **buildoutDegree:** (int) The number of degrees of relationships to show around
  each search entity
- **maxEntities:** (int) The maximum number of entities to return in the
  discovered network
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later

The functions return a JSON document that identifies the path between the
each set of search entities (if the path exists), and the information on the
entities in question (search entities, path entities, and build-out entities.

In [ ]:
StringBuffer response = new StringBuffer();
int maxDegree = 2;
int buildoutDegree = 1;
int maxEntities = 12;
String recordList = "{\"RECORDS\": [{\"RECORD_ID\": \""+recordID1+"\", \"DATA_SOURCE\": \""+dataSourceCode1+"\" }, {\"RECORD_ID\": \""+recordID2+"\", \"DATA_SOURCE\": \""+dataSourceCode2+"\" }]}";

int return_code = g2engine.findNetworkByRecordID(
                            recordList, 
                            maxDegree, 
                            buildoutDegree, 
                            maxEntities, 
                            response);
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

## Connection details

### findInterestingEntities

`findInterestingEntitiesByEntityID` and `findInterestingEntitiesByRecordID` are early adopter features, contact support for their use.

The `whyEntities()`, `whyRecords()`, `whyEntityByRecordID()`,`whyEntityByEntityID()`, and `howEntityByEntityID()` functions can be used
to determine why and how records belong to their resolved entities.
These functions will compare the record data within an entity against the
rest of the entity data, and show why they are connected.
This is calculated based on the features that record data represents.

###### Parameters

- **dataSourceCode:** (str) The data source for the record to be analyzed
- **entityID:** (int) The entity ID for the entity to be analyzed
- **recordID:** (str) The record ID for the record to be analyzed
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later
- **g2EngineFlags:** (long) Operational flags

The function returns a JSON document that gives the results of the record
analysis.
The document contains a section called "WHY_RESULTS",
which shows how specific records relate to the rest of the entity.
It has a "WHY_KEY", which is similar to a match key, in defining the relevant
connected data.
It shows candidate keys for features that initially cause the records
to be analyzed for a relationship,
plus a series of feature scores that show how similar the feature data was.

The response document also contains a separate ENTITIES section,
with the full information about the resolved entity.
(Note: When working with this entity data,
Senzing recommends using the flags `G2_ENTITY_OPTION_INCLUDE_INTERNAL_FEATURES`
and `G2_ENTITY_OPTION_INCLUDE_FEATURE_STATS`.
This will provide detailed feature data that is not included by default,
but is useful for understanding the WHY_RESULTS data.)

### whyEntities


###### Parameters

- **entityID1:** (int) The entity ID for the first entity to be analyzed
- **entityID2:** (int) The entity ID for the second entity to be analyzed
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.whyEntities(
                            entityID1,
                            entityID2,
                            response);
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);       

### whyRecords

###### Parameters

- **dataSourceCode1:** (str) The name of the first data source the record
  is associated with.
  This value is configurable to the system
- **recordID1:** (int) The record ID for the first record to be analyzed
- **dataSourceCode2:** (str) The name of the second data source the record
  is associated with.
  This value is configurable to the system
- **recordID2:** (int) The record ID for the second record to be analyzed
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.whyRecords(
                            dataSourceCode1,
                            recordID1,
                            dataSourceCode2,
                            recordID2,
                            response);
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);       

### whyEntityByEntityID

###### Parameters

- **entityID:** (int) The entity ID for the entity to be analyzed
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();


int return_code = g2engine.whyEntityByEntityID(entityID1,response);


if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

### whyEntityByRecordID

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record
  is associated with.
  This value is configurable to the system
- **recordID:** (int) The record ID for the record to be analyzed
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.whyEntityByRecordID(
                            dataSourceCode1,
                            recordID1,
                            response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

### howEntityByEntityID

###### Parameters

- **entity_id:** (int) The entity ID for the entity to be analyzed
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

Create some records to show resolution steps

In [ ]:
String data1 =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Joe\", \"NAME_LAST\": \"Schmoe\", \"SSN_NUMBER\": \"111-22-3456\", \"DOB\": \"03-22-1982\"}";
String data2 =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Joe\", \"NAME_LAST\": \"Schmoe\", \"SSN_NUMBER\": \"111-22-3456\", \"CITIZENSHIP\": \"British\"}";
String data3 =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Joe\", \"NAME_LAST\": \"Schmoe\", \"SSN_NUMBER\": \"111-22-3456\", \"NATIONALITY\": \"Australian\"}";

String recordID1 = "10";
String recordID2 = "11";
String recordID3 = "12";
String dataSourceCode = "TEST";

g2engine.addRecord(dataSourceCode, recordID1, data1, loadID);
g2engine.addRecord(dataSourceCode, recordID2, data2, loadID);
g2engine.addRecord(dataSourceCode, recordID3, data3, loadID);

StringBuffer response = new StringBuffer();

g2engine.getEntityByRecordID(dataSourceCode, recordID1, g2EngineFlags, response);
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.howEntityByEntityID(
                            entityID,
                            response);
                            
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

## Reporting

Exporting entity data from resolved entities is one of the core purposes of
Senzing software.
In just a few short steps, the Senzing engine allows users to export entity
data in either JSON or CSV format.

### exportJSONEntityReport

There are three steps to exporting resolved entity data from the G2Engine
object in JSON format.
First, use the `exportJSONEntityReport()` method to generate the `exportHandle`.

###### Parameters

- **g2EngineFlags**: (int) An integer specifying which entity details should be
  included in the export.
- **exportHandle:** (long) A long from which resolved entity data may be

In [ ]:
StringBuffer response = new StringBuffer();
Result<Long> exportHandle = new Result<Long>();

long g2EngineFlags = g2engine.G2_EXPORT_DEFAULT_FLAGS;
int return_code = g2engine.exportJSONEntityReport(g2EngineFlags, exportHandle);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code);

### fetchNext

Second, use the `fetchNext()` method to read the exportHandle and export a
row of JSON output containing the entity data for a single entity.
Note that successive calls of `fetchNext()` will export successive rows of
entity data.

###### Parameters

- **exportHandle:** (long) A long from which resolved entity data may be
  read and exported.
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later

In [ ]:
while(true){
    StringBuffer response = new StringBuffer();
    int return_code = g2engine.fetchNext(exportHandle.getValue(), response);
    if(return_code != 0){
        System.out.print(g2engine.getLastException());
        break;
    }
    if(response.length() == 0){
        break;
    }
    System.out.println(response.toString());
}

### closeExport

###### Parameters

- **exportHandle:** (long) A long from which resolved entity data may be
  read and exported.

In [ ]:
int return_code = g2engine.closeExport(exportHandle.getValue());

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code);

### exportCSVEntityReport

There are three steps to exporting resolved entity data from the G2Engine
object in CSV format.
First, use the `exportCSVEntityReport()` method to generate a long integer,
referred to here as an 'exportHandle'.

###### Parameters

- **headers:** (str) A comma-separated list of column names for the CSV
  export. (These are listed a little further down.)
- **g2EngineFlags:** (long) An integer specifying which entity details should be
  included in the export.
  See the "Entity Export Flags" section in the link below for further details.
- **exportHandle:** (long) A long from which resolved entity data may be
  read and exported.
  
Second, use the `fetchNext()` method to read the exportHandle and export a
row of CSV output containing the entity data for a single entity.
Note that the first call of `fetchNext()` will yield a header row,
and that successive calls of `fetchNext()` will export successive rows of
entity data.

In [ ]:
String headers = "RESOLVED_ENTITY_ID,RESOLVED_ENTITY_NAME,RELATED_ENTITY_ID,MATCH_LEVEL,MATCH_KEY,IS_DISCLOSED,IS_AMBIGUOUS,DATA_SOURCE,RECORD_ID,JSON_DATA,LAST_SEEN_DT,NAME_DATA,ATTRIBUTE_DATA,IDENTIFIER_DATA,ADDRESS_DATA,PHONE_DATA,RELATIONSHIP_DATA,ENTITY_DATA,OTHER_DATA";
Result<Long> exportHandle = new Result<Long>();
StringBuffer response = new StringBuffer();

g2engine.exportCSVEntityReport(headers, g2EngineFlags, exportHandle);

while(true){
    StringBuffer response = new StringBuffer();
    int return_code = g2engine.fetchNext(exportHandle.getValue(), response);
    if(return_code != 0){
        System.out.print(g2engine.getLastException());
        break;
    }
    if(response.length() == 0){
        break;
    }
    System.out.println(response.toString());
}
g2engine.closeExport(exportHandle.getValue());

## Cleanup

### purgeRepository

**Before** calling `purgeRepository()` all other instances of the Senzing API (whether in custom code, REST API, stream-loader, redoer, G2Loader, etc) **MUST** be destroyed or shutdown.

In [ ]:
int return_code= g2engine.purgeRepository();
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code)

### destroy
Once all searching is done in a process call `destroy()` to uninitialize Senzing and clean up resources. You should always do this once at the end of each process.

In [ ]:
int return_code = g2engine.destroy();

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code);